In [1]:
import pandas as pd
# Read in data and display first 5 rows
features = pd.read_csv('rba-dataset.csv')
features.head()

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,2020-02-03 12:43:30.772,-4324475583306591935,NaN,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,2020-02-03 12:43:43.549,-4324475583306591935,NaN,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2,2020-02-03 12:43:55.873,-3284137479262433373,NaN,81.167.144.58,NO,Vestland,Urangsvag,29695,Mozilla/5.0 (iPad; CPU OS 7_1 like Mac OS X) ...,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,3,2020-02-03 12:43:56.180,-4324475583306591935,NaN,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,2020-02-03 12:43:59.396,-4618854071942621186,NaN,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


In [2]:
features.isnull().sum()

index                              0
Login Timestamp                    0
User ID                            0
Round-Trip Time [ms]        29993329
IP Address                         0
Country                            0
Region                         47409
City                            8590
ASN                                0
User Agent String                  0
Browser Name and Version           0
OS Name and Version                0
Device Type                     1526
Login Successful                   0
Is Attack IP                       0
Is Account Takeover                0
dtype: int64

## Preprocessing

In [3]:
# rename above columns to snake case
features = features.rename(columns={'Login Timestamp': 'login_timestamp', 'User ID': 'user_id', 'Round-Trip Time [ms]':'round_trip','Region':'region', 'City':'city', 'ASN':'asn', 'IP Address': 'ip_address', 'Country': 'country', 'User Agent String': 'user_agent_string','Device Type': 'device_type', 'Browser Name and Version': 'browser', 'Is Account Takeover':'is_account_takeover', 'OS Name and Version':'os_detail','Login Successful':'is_login_success','Is Attack IP':'is_attack_ip'})

In [4]:
# check lenght in column user_agent_string
features['length'] = features['user_agent_string'].apply(
    lambda row: min(len(row), len(row)) if isinstance(row, str) else None
)
print(features['length'].mean())

136.652141700553


In [5]:
# only keep rows with device type desktop
features = features[features.device_type == 'desktop']
# filter the DataFrame based on the length of column 'user_agent_string'
features = features[features['user_agent_string'].str.len() < 136]

In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6120533 entries, 9 to 31269254
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   index                int64  
 1   login_timestamp      object 
 2   user_id              int64  
 3   round_trip           float64
 4   ip_address           object 
 5   country              object 
 6   region               object 
 7   city                 object 
 8   asn                  int64  
 9   user_agent_string    object 
 10  browser              object 
 11  os_detail            object 
 12  device_type          object 
 13  is_login_success     bool   
 14  is_attack_ip         bool   
 15  is_account_takeover  bool   
 16  length               int64  
dtypes: bool(3), float64(1), int64(4), object(9)
memory usage: 718.0+ MB


In [7]:
# generate SHA512 Hash from user_id as m2m token
import hashlib

def generate_sha512_hash(user_id):
    sha512_hash = hashlib.sha512()
    sha512_hash.update(str(user_id).encode('utf-8'))
    return sha512_hash.hexdigest()

features['token'] = features['user_id'].apply(generate_sha512_hash)
features.head()

,index,login_timestamp,user_id,round_trip,ip_address,country,region,city,asn,user_agent_string,browser,os_detail,device_type,is_login_success,is_attack_ip,is_account_takeover,length,token
9,9,2020-02-03 12:44:19.071,-3065936140549856249,NaN,92.221.109.162,NO,Rogaland,Sandnes,29695,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 69.0.3497.17.28,Mac OS X 10.14.6,desktop,True,False,False,126,4ffe29f1960c24624ec2c36909f3b39cb8d59fa18515f4...
10,10,2020-02-03 12:44:19.260,5932501938287412564,NaN,84.48.61.135,NO,-,-,15659,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 86.0.4202,Mac OS X 10.14.6,desktop,True,False,False,119,ecee6cc95d3b047c8f796b8e772a468124b7ddb599a7a3...
11,11,2020-02-03 12:44:24.849,-9080829243863829585,NaN,156.52.48.83,NO,Viken,Gressvik,29695,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 79.0.3945.192.203,Mac OS X 10.14.6,desktop,True,False,False,128,44504e123fc411a21a7757013e207ad22235493c94814f...
16,16,2020-02-03 12:44:31.854,-4663943525943860871,NaN,81.166.109.168,NO,Rogaland,Haugesund,29695,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 86.0.4202,Mac OS X 10.14.6,desktop,True,False,False,119,234e8db63aef8801564821e5ce6dccdcbea0d3b1702498...
18,18,2020-02-03 12:44:33.093,-4324475583306591935,NaN,46.183.175.255,NO,-,-,197475,Mozilla/5.0 (X11; CrOS aarch64 11316.123.0) Ap...,Chrome 72.0.3626,Chrome OS 11316.123.0,desktop,False,False,False,116,92e489df0d494e90736d5022e65ed9db36ef590d10234a...


In [8]:
features.shape

(6120533, 18)

In [9]:
df = features
df.to_csv('rba-dataset-renamed-desktop.csv', index=False)